In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [0]:
#Libraries
import pandas as pd
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
from spacy.lang.pt.examples import sentences
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.decomposition import PCA
import pickle
#nlp = spacy.load('pt')
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE 
import matplotlib.pyplot as plt
#pip install scipy==0.10.1
import gensim
from gensim.models import KeyedVectors

from __future__ import division, print_function
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import text
from tensorflow.keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#Take Files From GoogleDrive
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Research/DataSets/')
path= '/content/drive/My Drive/Research/DataSets/'

In [0]:
%run '/content/drive/My Drive/Fraud detection in internal auditing/Colab Notebooks/NLP functions_Gus.ipynb'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def overSamplingRISKs(df,rate):
  X = df 
  not_HIGH = df[df.RISK!="HIGH"]
  HIGH = df[df.RISK=="HIGH"]
  oversample_by= int((rate*((len(df[df.RISK!="HIGH"])+len(df[df.RISK=="HIGH"]))-(len(df[df.RISK=="HIGH"]))))/(1-rate))
  # OVERsample minority
  high_upsampled = resample(HIGH,
                            replace=True, # sample with replacement
                            n_samples=oversample_by, # match number in majority class
                            random_state=27) # reproducible results

  upsampled = pd.concat([not_HIGH, high_upsampled])
  print( upsampled.RISK.value_counts() )

  return upsampled

In [0]:
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'
from sklearn.model_selection import train_test_split
import numpy as np
from keras.layers import MaxPooling1D

ele='2016_DF_cleaned_not_balanced.pkl'
filename = DATASETS_FOLDER+ele
df = pd.read_pickle(filename)
datatrain,datatest= train_test_split(df, test_size=0.30, random_state=42)
datatrain = overSamplingRISKs(datatrain,0.30)

MEDIUM-LOW    12467
HIGH           5343
Name: RISK, dtype: int64


In [0]:
MAX_SEQUENCE_LENGTH = 5000
EMBEDDING_DIM = 300
emdedding_size = 32

In [0]:
embeddings_index = {}
word_list=[]
f = open('glove_s300.txt', errors = 'ignore', encoding='utf-8')
print(f)
for line in f:
  values = line.split()
  word = ''.join(values[:-300])
  word_list.append(word)
  coefs = np.asarray(values[-300:], dtype='float32')
  embeddings_index[word] = coefs
f.close()


<_io.TextIOWrapper name='glove_s300.txt' mode='r' encoding='utf-8'>


In [0]:
%run '/content/drive/My Drive/Fraud detection in internal auditing/Colab Notebooks/NLP functions_Gus.ipynb'
# %run '/content/drive/My Drive/Colab Notebooks/NLP-4-Classification-FUNCTIONS.ipynb'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'

In [0]:
def dummy_encoder(df):
  df_dummies = pd.get_dummies(df['RISK'],prefix='Risk')
  df = pd.concat([df,df_dummies],axis=1)
  return df

In [0]:
tokenizer= text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(datatrain.sent_token_final.tolist())
training_sequences = tokenizer.texts_to_sequences(datatrain.sent_token_final.tolist())
train_word_index = tokenizer.word_index
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
ele_to_pop=[]
for word, index in train_word_index.items():
    try:
      embedding_vector = embeddings_index[word]
      train_embedding_weights[index] = embedding_vector
    except KeyError:
      train_embedding_weights[index] = np.zeros(EMBEDDING_DIM)

for name,key in list(train_word_index.items()):
  if name in ele_to_pop:
    del train_word_index[name]
label_names = ['Risk_HIGH', 'Risk_MEDIUM-LOW']

In [0]:
model = Sequential()
#using fasttext
model.add(Embedding(len(train_word_index), EMBEDDING_DIM, weights=[train_embedding_weights], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(Dropout(0.5))   
model.add(Bidirectional(LSTM(emdedding_size, return_sequences=True)))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(len(list(label_names)),activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
test_sequences = tokenizer.texts_to_sequences(datatest.sent_token_final.tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

x_train = train_cnn_data
y_train = datatrain[label_names].values
y_tr = y_train

x_test= test_cnn_data
y_test= datatest[label_names].values
y_te = y_test


In [0]:
model.fit(x_train,y_tr, batch_size=32, epochs=20,validation_data=(x_test,y_te))

Epoch 1/20
557/557 [==============================] - 217s 390ms/step - loss: 0.6156 - accuracy: 0.7679 - val_loss: 0.3456 - val_accuracy: 0.9074
Epoch 2/20
557/557 [==============================] - 216s 388ms/step - loss: 0.2354 - accuracy: 0.8973 - val_loss: 0.2777 - val_accuracy: 0.9152
Epoch 3/20
557/557 [==============================] - 216s 388ms/step - loss: 0.1624 - accuracy: 0.9343 - val_loss: 0.2237 - val_accuracy: 0.9382
Epoch 4/20
557/557 [==============================] - 215s 387ms/step - loss: 0.1246 - accuracy: 0.9537 - val_loss: 0.2328 - val_accuracy: 0.9577
Epoch 5/20
557/557 [==============================] - 215s 386ms/step - loss: 0.1061 - accuracy: 0.9618 - val_loss: 0.2531 - val_accuracy: 0.9487
Epoch 6/20
557/557 [==============================] - 215s 386ms/step - loss: 0.0765 - accuracy: 0.9726 - val_loss: 0.2701 - val_accuracy: 0.9566
Epoch 7/20
557/557 [==============================] - 215s 387ms/step - loss: 0.0699 - accuracy: 0.9742 - val_loss: 0.2513 -

In [0]:
predictions0 = model.predict(train_cnn_data, batch_size=32, verbose=1)
predictions = model.predict(test_cnn_data, batch_size=32, verbose=1)

173/173 [==============================] - 30s 172ms/step


In [0]:
predictions_01 = np.asarray([np.argmax(x) for x in predictions0])
predictions_1 = np.asarray([np.argmax(x) for x in predictions])

In [0]:
acc, pre, rec, f1s = get_metrics(datatrain.FinalLabel,predictions_01)
acc, pre, rec, f1s = get_metrics(datatest.FinalLabel,predictions_1)

Accuracy: 0.9943  Precision: 0.994  Recall: 0.9979  F1 Score: 0.996
Accuracy: 0.9602  Precision: 0.9685  Recall: 0.991  F1 Score: 0.9796
